<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2174, done.
remote: Counting objects: 100% (992/992), done.
remote: Compressing objects: 100% (534/534), done.
remote: Total 2174 (delta 643), reused 686 (delta 449), pack-reused 1182
Receiving objects: 100% (2174/2174), 52.25 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (1421/1421), done.


# import

In [28]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [29]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [30]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023,980


In [31]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023,70


In [32]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
140-023,10


# 1.試算模式

In [33]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [34]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [35]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


# 檢查三項約束條件

In [36]:
c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,69.163988


In [37]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

981.4707079837214

In [38]:
c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

9.062412707267518

In [39]:
c620_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [40]:
c660_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [41]:
c670_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

# 推薦模式

In [42]:
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

[I 2021-04-23 09:34:46,219] A new study created in memory with name: no-name-7b25bedf-85e4-44bf-9510-81b6db9e72d4


(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


[I 2021-04-23 09:34:46,263] Trial 0 finished with value: 4.69761798656819 and parameters: {'Density_Feed Properties': 0.8634799021058772, 'Density_Vent Gas Production Rate and Composition': 0.42835939101885784, 'Density_Distillate Production Rate and Composition': 0.7966623308665239, 'Density_Sidedraw Production Rate and Composition': 0.8804619578191734, 'Density_Bottoms Production Rate and Composition': 0.8718907244100118, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 129.19652855390788, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.42753132455039, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.368009009628, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.98095320666354, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 194.68814875890286}. Best is trial 0 with value: 4.69761798656819.
[I 2021-04-23 09:34:46,319] Trial 1 finished with val

(1, 3)
(1, 41)
(1, 10)
bz_error: 0.001024689118793276


[I 2021-04-23 09:35:09,532] Trial 2 finished with value: 70.68177620537244 and parameters: {'Density_Feed Properties': 0.8762708931294649, 'Density_Vent Gas Production Rate and Composition': 0.5445964118763696, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8669976091656204, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8836921949228048, 'Density_Bottoms Production Rate and Composition': 0.8707615403191946, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 119.45758810356698, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.683145138632788, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 10.896309225097694, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.52807837514423, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 89.6343336228705, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 979.9999984271551}. Be

(1, 2)
(1, 41)
(1, 10)
['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw', 'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']
['Combined Feed to Benzene Column C660_Detailed Composition_Hydrogen_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Methane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Ethane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Propane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_n-Butane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_n-Pentane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_n-Hexane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Benzene_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Cyclohexane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_n-Heptane_wt%', 'Combined Feed to Benzene Column C660_Detailed Composition_Water_wt%', 'Combin

In [43]:
bz_error,nainbz_error,tol_error

(0.001024689118793276, 17.23124013230904, 0.4949964240189626)

# 計算delta調幅

In [44]:
c620_op2-c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
140-023,-0.001007,-0.002139,0.005481,0.001454,-0.000094,-8.066031,-0.829219,-1.089793,-1.675406,0.077741


In [45]:
c660_op2-c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
140-023,-0.000013,0.000011,-0.000061,0.000052,-0.000006,0.000003,0.000006,0.000053,0.000031,-0.000027


In [46]:
c670_op2-c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
140-023,5.960464e-08,-0.000013,1.30864,0.088829,0.057325,-0.016434,0.071548


# 檢查三項約束條件

In [47]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,69.998975


In [48]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

962.768759867691

In [49]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

10.494996424018963

In [50]:
c620_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [51]:
c660_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [52]:
c670_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64